In [12]:
# Cellule 1 : Configuration initiale et imports
import sys
from pathlib import Path
import pandas as pd
import torch
import sacrebleu

# 1. Configuration robuste du chemin racine
ROOT_DIR = Path("..").resolve()
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

# 2. Rechargement automatique des modules src/
%load_ext autoreload
%autoreload 2

# 3. Imports de notre backend
from src.translation_models import Encoder, Decoder, Seq2SeqVanilla
from src.translation_utils import train_sentencepiece_models, TranslationDataset, collate_fn_translation
from src.train_utils import train_model

# 4. Definition du hardware
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Racine du projet : {ROOT_DIR}")
print(f"Device utilise : {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Racine du projet : C:\Users\benic\Documents\Projet_DL_Translation
Device utilise : cpu


In [7]:
# Cellule 2 : Entrainement des Tokenizers (BPE)
# Attention : Cette etape ne doit voir QUE le set de Train !
train_csv_path = str(ROOT_DIR / 'data' / 'processed' / 'train_nmt_fr_en.csv')
models_dir_path = str(ROOT_DIR / 'models')

print("Creation des dictionnaires de sous-mots...")
train_sentencepiece_models(train_csv_path, vocab_size=8000, model_dir=models_dir_path)

Creation des dictionnaires de sous-mots...
Chargement des donnees d'entrainement depuis C:\Users\benic\Documents\Projet_DL_Translation\data\processed\train_nmt_fr_en.csv
Entrainement du modele SentencePiece FR...
Entrainement du modele SentencePiece EN...
Entrainement SentencePiece termine avec succes. Fichiers sauvegardes dans : C:\Users\benic\Documents\Projet_DL_Translation\models


In [13]:
# Cellule 3 : Chargement des Datasets (MODE TEST 20%)
from torch.utils.data import DataLoader

spm_fr_path = str(ROOT_DIR / 'models' / 'spm_fr.model')
spm_en_path = str(ROOT_DIR / 'models' / 'spm_en.model')

valid_csv_path = str(ROOT_DIR / 'data' / 'processed' / 'valid_nmt_fr_en.csv')

# On ajoute l'argument sample_frac=0.2 pour ne prendre que 20% des CSV
train_data = TranslationDataset(train_csv_path, spm_fr_path, spm_en_path, sample_frac=0.2)
valid_data = TranslationDataset(valid_csv_path, spm_fr_path, spm_en_path, sample_frac=0.2)

BATCH_SIZE = 32

train_iterator = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn_translation)
valid_iterator = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn_translation)

print(f"Nombre de batches pour l'entrainement (20%) : {len(train_iterator)}")
print(f"Nombre de batches pour la validation (20%) : {len(valid_iterator)}")

Dataset reduit a 20.0% : 6011 phrases chargees.
Dataset reduit a 20.0% : 207 phrases chargees.
Nombre de batches pour l'entrainement (20%) : 188
Nombre de batches pour la validation (20%) : 7


In [14]:
# Cellule 4 : Instanciation et Entrainement du modele Vanilla Seq2Seq
import torch.optim as optim
import torch.nn as nn

# Hyperparametres
INPUT_DIM = 8000
OUTPUT_DIM = 8000
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
DROPOUT = 0.5

# Creation de l'architecture
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DROPOUT)
model = Seq2SeqVanilla(enc, dec, device).to(device)

# Optimiseur et Fonction de perte (on ignore l'ID du padding pour ne pas fausser la loss)
optimizer = optim.Adam(model.parameters(), lr=0.001)
PAD_IDX = 0 # sp_en.pad_id()
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

N_EPOCHS = 10
CLIP = 1.0
SAVE_PATH = str(ROOT_DIR / 'models' / 'nmt_baseline_vanilla.pt')

# Lancement de la boucle avec Early Stopping manuel base sur la Validation Loss
train_model(model, train_iterator, valid_iterator, optimizer, criterion, train_data.sp_en, N_EPOCHS, CLIP, SAVE_PATH, device)

Demarrage de l'entrainement pour 10 epoques...


Epoque: 01 | Temps: 7.0m 47s => Checkpoint sauvegarde !
	Train Loss: 6.527 | Train PPL: 683.532
	 Val. Loss: 6.382 |  Val. PPL: 591.096
	Metriques Val -> BLEU: 1.52 | chrF: 4.80 | METEOR: 0.00



Epoque: 02 | Temps: 7.0m 27s 
	Train Loss: 6.123 | Train PPL: 456.235
	 Val. Loss: 6.440 |  Val. PPL: 626.490
	Metriques Val -> BLEU: 7.15 | chrF: 10.17 | METEOR: 0.00



Epoque: 03 | Temps: 7.0m 48s 
	Train Loss: 5.986 | Train PPL: 397.930
	 Val. Loss: 6.444 |  Val. PPL: 628.853
	Metriques Val -> BLEU: 15.11 | chrF: 15.87 | METEOR: 2.26



Epoque: 04 | Temps: 7.0m 44s 
	Train Loss: 5.874 | Train PPL: 355.742
	 Val. Loss: 6.525 |  Val. PPL: 682.282
	Metriques Val -> BLEU: 28.12 | chrF: 14.13 | METEOR: 2.63



...

Epoque: 06 | Temps: 10.0m 49s 
	Train Loss: 5.730 | Train PPL: 307.825
	 Val. Loss: 6.471 |  Val. PPL: 645.991
	Metriques Val -> BLEU: 2.44 | chrF: 8.50 | METEOR: 1.11



Epoque: 07 | Temps: 7.0m 45s 
	Train Loss: 5.668 | Train PPL: 289.582
	 Val. Loss: 6.423 |  Val. PPL: 615.560
	Metriques Val -> BLEU: 1.52 | chrF: 1.60 | METEOR: 0.00



Epoque: 08 | Temps: 8.0m 21s 
	Train Loss: 5.607 | Train PPL: 272.233
	 Val. Loss: 6.447 |  Val. PPL: 630.643
	Metriques Val -> BLEU: 1.91 | chrF: 3.52 | METEOR: 0.00



Epoque: 09 | Temps: 11.0m 31s 
	Train Loss: 5.561 | Train PPL: 260.029
	 Val. Loss: 6.461 |  Val. PPL: 639.395
	Metriques Val -> BLEU: 11.21 | chrF: 11.29 | METEOR: 1.94



Epoque: 10 | Temps: 11.0m 12s => Checkpoint sauvegarde !
	Train Loss: 5.466 | Train PPL: 236.605
	 Val. Loss: 6.382 |  Val. PPL: 591.028
	Metriques Val -> BLEU: 4.05 | chrF: 12.49 | METEOR: 1.54

Entrainement termine. Meilleur modele sauvegarde sous : C:\Users\benic\Documents\Projet_DL_Translation\models\nmt_baseline_vanilla.pt


In [15]:
# Cellule 5 : Inference sur le set de TEST (Scelle jusqu'a present)
# Cette cellule calcule les predictions et les stocke. Elle est longue, ne la lancer qu'une fois !
import sentencepiece as spm

# Chargement strict du meilleur checkpoint
model.load_state_dict(torch.load(SAVE_PATH, map_location=device))
model.eval()

# Chargement direct de SentencePiece pour l'inference pure
sp_fr = spm.SentencePieceProcessor()
sp_fr.load(spm_fr_path)
sp_en = spm.SentencePieceProcessor()
sp_en.load(spm_en_path)

test_csv_path = str(ROOT_DIR / 'data' / 'processed' / 'test_nmt_fr_en.csv')
df_test = pd.read_csv(test_csv_path)
predictions = []

def translate_sentence(sentence, model, sp_fr, sp_en, device, max_len=50):
    model.eval()
    tokens = [sp_fr.bos_id()] + sp_fr.encode_as_ids(sentence) + [sp_fr.eos_id()]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        
    trg_indexes = [sp_en.bos_id()]
    
    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        if pred_token == sp_en.eos_id():
            break
            
    trg_indexes = [t for t in trg_indexes if t not in [sp_en.bos_id(), sp_en.eos_id(), sp_en.pad_id()]]
    return sp_en.decode_ids(trg_indexes)

print("Demarrage de la traduction du set de Test...")
for idx, row in df_test.iterrows():
    pred = translate_sentence(str(row['text_fr']), model, sp_fr, sp_en, device)
    predictions.append(pred)

df_test['prediction'] = predictions
print("Inference terminee ! Resultats stockes dans le DataFrame.")

Demarrage de la traduction du set de Test...
Inference terminee ! Resultats stockes dans le DataFrame.


In [16]:
# Cellule 6 : Tableau comparatif
# Vous pouvez relancer cette cellule autant de fois que vous voulez pour voir d'autres exemples
pd.set_option('display.max_colwidth', None)
df_test[['text_fr', 'text_en', 'prediction']].sample(10)

,text_fr,text_en,prediction
653,"En réalité, la composition des roches, va nous permettre de savoir dans quelles conditions elle s'est formée.","In fact, the composition of the rocks helps us to know the conditions in which they were formed.","The,,,,,,,,"
99,"Comme nous l'a dit Darwin, s'adapter ou mourir.","As Darwin told us, we should adapt or perish.","The,,,,,,,,"
987,Et il n'a pas eu de chance.,And was unfortunate.,"And then, I,,,,,,"
954,"Première idée fausse : ça coûte super cher de traverser ; avec tout cet argent, ils n'auraient pas pu faire autre chose ?","First misconception: passing through is expensive; with all this money, couldn't they do something else?","The,,,,,,,,"
309,C'est la seule femme du groupe s'il vous plaît.,She is the only woman in the band.,"It's a, of the,"
501,Je suis fier de moi. »,"I'm proud of myself.""",I'm going to to
240,"Le septième continent, il est dans la mer, c'est des particules de plastique qui, petit à petit, ruissellent et vont créer la plus grande décharge mondiale.","The seventh continent is in the sea, made of plastic particles that flow slowly, and will create the biggest garbage dump in the world.","The,,,,,,,,"
977,"Il m'a dit : « Chaque jour, je vais y retourner parce qu'ici, je suis déjà mort. »","He told me, ""Each day, I will go back because here I am already dead.""","It's a, of the,"
528,Vous me direz : « Certains sont plus doués que d'autres. »,"You may say, ""Some people are more gifted than others.""","So, I,,,,,,"
687,"C'est formidable, non ?","It's amazing, isn't it?","It's a, of the,"


In [17]:
# Cellule 7 : Preuve mathematique de la limite du modele Vanilla
# Calcul des longueurs des phrases sources
df_test['src_len'] = df_test['text_fr'].apply(lambda x: len(str(x).split()))

df_short = df_test[df_test['src_len'] < 10]
df_long = df_test[df_test['src_len'] > 25]

def compute_bleu(df):
    if len(df) == 0: return 0.0
    preds = df['prediction'].tolist()
    # SacreBLEU attend une liste de listes pour les references
    refs = [df['text_en'].tolist()]
    return sacrebleu.corpus_bleu(preds, refs).score

print(f"--- ANALYSE DE ROBUSTESSE ---")
print(f"BLEU Global : {compute_bleu(df_test):.2f}")
print(f"BLEU Phrases Courtes (<10 mots) : {compute_bleu(df_short):.2f} (Support: {len(df_short)})")
print(f"BLEU Phrases Longues (>25 mots) : {compute_bleu(df_long):.2f} (Support: {len(df_long)})")
print("\nConclusion: L'effondrement du score sur les longues phrases demontre la perte d'information dans le vecteur de contexte unique.")

--- ANALYSE DE ROBUSTESSE ---
BLEU Global : 0.18
BLEU Phrases Courtes (<10 mots) : 0.25 (Support: 394)
BLEU Phrases Longues (>25 mots) : 0.03 (Support: 152)

Conclusion: L'effondrement du score sur les longues phrases demontre la perte d'information dans le vecteur de contexte unique.
